CS4001/4042 Assignment 1, Part B, Q4
---

Model degradation is a common issue faced when deploying machine learning models (including neural networks) in the real world. New data points could exhibit a different pattern from older data points due to factors such as changes in government policy or market sentiments. For instance, housing prices in Singapore have been increasing and the Singapore government has introduced 3 rounds of cooling measures over the past years (16 December 2021, 30 September 2022, 27 April 2023).

In such situations, the distribution of the new data points could differ from the original data distribution which the models were trained on. Recall that machine learning models often work with the assumption that the test distribution should be similar to train distribution. When this assumption is violated, model performance will be adversely impacted.  In the last part of this assignment, we will investigate to what extent model degradation has occurred.




---



---



Your co-investigators used a linear regression model to rapidly test out several combinations of train/test splits and shared with you their findings in a brief report attached in Appendix A below. You wish to investigate whether your deep learning model corroborates with their findings.

In [1]:
!pip install alibi-detect

     |████████████████████████████████| 372 kB 1.4 MB/s eta 0:00:01
     |████████████████████████████████| 2.5 MB 21.5 MB/s eta 0:00:01
     |████████████████████████████████| 3.4 MB 29.0 MB/s eta 0:00:01
     |████████████████████████████████| 54.7 MB 555 kB/s  eta 0:00:01
     |████████████████████████████████| 7.7 MB 19.8 MB/s eta 0:00:01
     |████████████████████████████████| 115 kB 14.5 MB/s eta 0:00:01
     |████████████████████████████████| 2.9 MB 18.2 MB/s eta 0:00:01
     |████████████████████████████████| 13.0 MB 20.0 MB/s eta 0:00:01
  Using cached toml-0.10.2-py2.py3-none-any.whl (16 kB)
     |████████████████████████████████| 30.4 MB 136 kB/s eta 0:00:01
     |████████████████████████████████| 19.8 MB 143 kB/s eta 0:00:011
  Using cached lazy_loader-0.3-py3-none-any.whl (9.1 kB)
     |████████████████████████████████| 222 kB 24.0 MB/s eta 0:00:01
     |████████████████████████████████| 313 kB 30.7 MB/s eta 0:00:01
  Using cached PyWavelets-1.4.1-cp39-cp39-macosx_10_13_x8

In [5]:
SEED = 42

import os

import random
random.seed(SEED)

import numpy as np
np.random.seed(SEED)

import pandas as pd
import pytorch_tabular
import math
from sklearn.metrics import r2_score, mean_squared_error
from alibi_detect.cd import TabularDrift

> Evaluate your model from B1 on data from year 2022 and report the test R2.

In [10]:
df = pd.read_csv('hdb_price_prediction.csv')

# TODO: Enter your code here
# model is in saved_models/b1.ckpt

model = pytorch_tabular.tabular_model.TabularModel.load_model('saved_models/b1')

test_data = df[df['year'] == 2022]
predictions = model.predict(test_data)

# print("RMSE: ", math.sqrt(mean_squared_error(test_data['resale_price'], predictions['resale_price_prediction'])))
print("R2: ", r2_score(test_data['resale_price'], predictions['resale_price_prediction']))


2023-10-10 15:02:52,166 - {pytorch_tabular.tabular_model:129} - INFO - Experiment Tracking is turned off
2023-10-10 15:02:52,168 - {pytorch_tabular.tabular_model:268} - INFO - Preparing the Trainer
/Users/advait/Desktop/gitpositories/SC4001-Assignment/env/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:589: LightningDeprecationWarning: The Trainer argument `auto_select_gpus` has been deprecated in v1.9.0 and will be removed in v2.0.0. Please use the function `pytorch_lightning.accelerators.find_usable_cuda_devices` instead.
  rank_zero_deprecation(
Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.rich_model_summary.RichModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Output()

R2:  0.4388472512009519


> Evaluate your model from B1 on data from year 2023 and report the test R2.

In [11]:
# TODO: Enter your code here

test_data = df[df['year'] == 2023]
predictions = model.predict(test_data)

print("R2: ", r2_score(test_data['resale_price'], predictions['resale_price_prediction']))

Output()

R2:  0.16212590443081432


> Did model degradation occur for the deep learning model?


As observed from the decrease in R2. We can observe that model degradatoin has occurred for this model



---



---



Model degradation could be caused by [various data distribution shifts](https://huyenchip.com/2022/02/07/data-distribution-shifts-and-monitoring.html#data-shift-types): covariate shift (features), label shift and/or concept drift (altered relationship between features and labels).
There are various conflicting terminologies in the [literature](https://www.sciencedirect.com/science/article/pii/S0950705122002854#tbl1). Let’s stick to this reference for this assignment.

> Using the **Alibi Detect** library, apply the **TabularDrift** function with the training data (year 2019 and before) used as the reference and **detect which features have drifted** in the 2023 test dataset. Before running the statistical tests, ensure you **sample 1000 data points** each from the train and test data. Do not use the whole train/test data. (Hint: use this example as a guide https://docs.seldon.io/projects/alibi-detect/en/stable/examples/cd_chi2ks_adult.html)


In [44]:
target = ['resale_price']
categorical_cols = ['month', 'town', 'flat_model_type', 'storey_range']
continuous_cols = ['dist_to_nearest_stn', 'dist_to_dhoby', 'degree_centrality', 'eigenvector_centrality', 'remaining_lease_years', 'floor_area_sqm']

# Extract unique categories for each categorical column
category_map = {}
for i, col in enumerate(categorical_cols):
    category_map[i] = df[col].unique().tolist()

n_ref = 1000
n_test = 1000

# Splitting the dataset into train (reference) and test based on the 'year' column
X_train = df[df['year'] <= 2019]
X_test = df[df['year'] == 2023]

# Limiting the number of rows for both reference and test datasets
X_train = X_train[:n_ref]
X_test = X_test[:n_test]

X_ref = X_train[categorical_cols + continuous_cols].values
X_test = X_test[categorical_cols + continuous_cols].values

y_ref = X_train[target].values
y_test = df[df['year'] == 2023][target].values[:n_test]

categories_per_feature = {f: None for f in list(category_map.keys())}
cd = TabularDrift(X_ref, p_val=.05, categories_per_feature=categories_per_feature)

In [46]:
predictions = cd.predict(X_test)
labels = ['No','Yes']
print('Drift? {}'.format(labels[predictions['data']['is_drift']]))

fpreds = cd.predict(X_test, drift_type='feature')

for f in range(cd.n_features):
    stat = 'Chi2' if f in list(categories_per_feature.keys()) else 'K-S'
    fname = feature_names[f]
    is_drift = fpreds['data']['is_drift'][f]
    stat_val, p_val = fpreds['data']['distance'][f], fpreds['data']['p_val'][f]
    print(f'{fname} -- Drift? {labels[is_drift]} -- {stat} {stat_val:.3f} -- p-value {p_val:.3f}')

Drift? Yes
month -- Drift? No -- Chi2 0.000 -- p-value 1.000
town -- Drift? Yes -- Chi2 667.474 -- p-value 0.000
flat_model_type -- Drift? Yes -- Chi2 77.586 -- p-value 0.000
storey_range -- Drift? Yes -- Chi2 38.800 -- p-value 0.001
dist_to_nearest_stn -- Drift? No -- K-S 0.055 -- p-value 0.094
dist_to_dhoby -- Drift? Yes -- K-S 0.218 -- p-value 0.000
degree_centrality -- Drift? No -- K-S 0.029 -- p-value 0.783
eigenvector_centrality -- Drift? Yes -- K-S 0.195 -- p-value 0.000
remaining_lease_years -- Drift? Yes -- K-S 0.271 -- p-value 0.000
floor_area_sqm -- Drift? Yes -- K-S 0.134 -- p-value 0.000


> Assuming that the flurry of housing measures have made an impact on the relationship between all the features and resale_price (i.e. P(Y|X) changes), which type of data distribution shift possibly led to model degradation?


Concept Drift. This is because if the measures change the relationship between all the features and the resale_price, this would impact the conditions by which X impacts Y, thereby changing P(Y|X). This represents Concept Drift.

> From your analysis via TabularDrift, which features contribute to this shift?


- town
- flat_model_type
- storey_range
- dist_to_dhoby
- eigenvector_centrality
- remaining_lease_years
- floor_area_sqm

> Suggest 1 way to address model degradation and implement it, showing improved test R2 for year 2023.


The model used was trained on data from 2019 and before. Hence one way to address the model degradation is to train it on updated data. 

In [47]:
model.fit(train=df[df['year'] <= 2022])

Global seed set to 42
2023-10-10 16:23:16,870 - {pytorch_tabular.tabular_model:473} - INFO - Preparing the DataLoaders
2023-10-10 16:23:16,881 - {pytorch_tabular.tabular_datamodule:290} - INFO - Setting up the datamodule for regression task
2023-10-10 16:23:17,039 - {pytorch_tabular.tabular_model:521} - INFO - Preparing the Model: CategoryEmbeddingModel
2023-10-10 16:23:17,070 - {pytorch_tabular.tabular_model:268} - INFO - Preparing the Trainer
/Users/advait/Desktop/gitpositories/SC4001-Assignment/env/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:589: LightningDeprecationWarning: The Trainer argument `auto_select_gpus` has been deprecated in v1.9.0 and will be removed in v2.0.0. Please use the function `pytorch_lightning.accelerators.find_usable_cuda_devices` instead.
  rank_zero_deprecation(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
2023-

Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

/Users/advait/Desktop/gitpositories/SC4001-Assignment/env/lib/python3.9/site-packages/pytorch_lightning/core/module.py:493: UserWarning: You called `self.log('train_loss', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`
  rank_zero_warn(
/Users/advait/Desktop/gitpositories/SC4001-Assignment/env/lib/python3.9/site-packages/pytorch_lightning/core/module.py:493: UserWarning: You called `self.log('train_mean_squared_error', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`
  rank_zero_warn(
`Trainer.fit` stopped: `max_steps=100` reached.
Learning rate set to 0.5754399373371567
Restoring states from the checkpoint path at /Users/advait/Desktop/gitpositories/SC4001-Assignment/.lr_find_39a96633-fd70-4b0a-8742-aabf70fbc156.ckpt
Restored all states from the checkpoint file at /Users/advait/Desktop/gitpositories/SC4001-Assignment/.lr_find_39a96633-fd70-4b0a-8742-aabf70fbc156.ck

┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name             ┃ Type                      ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ _backbone        │ CategoryEmbeddingBackbone │  3.0 K │
│ 1 │ _embedding_layer │ Embedding1dLayer          │  1.6 K │
│ 2 │ head             │ LinearHead                │     51 │
│ 3 │ loss             │ MSELoss                   │      0 │
└───┴──────────────────┴───────────────────────────┴────────┘

Trainable params: 4.6 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 4.6 K                                                                                                
Total estimated model params size (MB): 0

Output()

2023-10-10 16:25:38,355 - {pytorch_tabular.tabular_model:584} - INFO - Training the model completed
2023-10-10 16:25:38,359 - {pytorch_tabular.tabular_model:1258} - INFO - Loading the best model
/Users/advait/Desktop/gitpositories/SC4001-Assignment/env/lib/python3.9/site-packages/pytorch_lightning/utilities/cloud_io.py:33: LightningDeprecationWarning: `pytorch_lightning.utilities.cloud_io.get_filesystem` has been deprecated in v1.8.0 and will be removed in v2.0.0. Please use `lightning_fabric.utilities.cloud_io.get_filesystem` instead.
  rank_zero_deprecation(


In [48]:
test_data = df[df['year'] == 2023]

predictions = model.predict(test_data)

print("R2: ", r2_score(test_data['resale_price'], predictions['resale_price_prediction']))


Output()

R2:  0.5254131746577532


### Appendix A



Here are our results from a linear regression model. We used StandardScaler for continuous variables and OneHotEncoder for categorical variables.

While 2021 data can be predicted well, test R2 dropped rapidly for 2022 and 2023.

| Training set | Test set | Test R2 |
|--------------|----------|---------|
| Year <= 2020 | 2021     | 0.76    |
| Year <= 2020 | **2022**     | 0.41    |
| Year <= 2020 | **2023**     | **0.10**   |



Similarly, a model trained on 2017 data can predict 2018-2021 well (with slight degradation in performance for 2021), but drops drastically in 2022 and 2023.

| Training set | Test set | Test R2 |
|--------------|----------|---------|
| 2017         | 2018     | 0.90    |
|              | 2019     | 0.89    |
|              | 2020     | 0.87    |
|              | 2021     | 0.72    |
|              | **2022**     | **0.37**    |
|              | **2023**     | **0.09**    |

With the test set fixed at year 2021, training on data from 2017-2020 still works well on the test data, with minimal degradation. Training sets closer to year 2021 generally do better.

| Training set | Test set | Test R2 |
|--------------|----------|---------|
| 2020         | 2021     | 0.81    |
| 2019         | 2021     | 0.75    |
| 2018         | 2021     | 0.73    |
| 2017         | 2021     | 0.72    |